In [1]:
# Libraries import
import sys
import os
import os.path as pth
sys.path.insert(0, '../..')
    
from lcav.lca_problem import *
from lcav.helpers import *
from lcav.io.configuration import *
from lcav.gui.plots import *
import matplotlib.pyplot as plt 
from premise import clear_cache

# Pretty print for Sympy
from sympy import init_printing
init_printing()

# Paths
#os.chdir('./notebook')
DATA_FOLDER = './data'
CONFIGURATION_FILE = pth.join(DATA_FOLDER, 'configuration_methodo_ei391.yaml')

#clear_cache()  # clear premise cache

In [2]:
_, model, methods = LCAProblemConfigurator(CONFIGURATION_FILE).generate()

In [3]:
#agb.findActivity('carbon monoxide, from RWGS, for Fischer Tropsch process, hydrogen from electrolysis', db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050', loc='World').listExchanges()
#test = agb.findActivity('kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation\n(remind_SSP2_Base_2020)', 
#                 db_name = 'Foreground DB')#.listExchanges()

#test = agb.findActivity('electricity production, photovoltaic, commercial', db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050', single=False)
#test = agb.findActivity('carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with heat pump heat, and grid electricity', 
#                        db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050',
#                        loc='World',
#                        single=False)

test = agb.findActivity('hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity', 
                        db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2020',
                        loc='World',
                        single=False)

test = agb.findActivity('transport, passenger aircraft, long haul', 
                        db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050',
                        #loc='World',
                        single=False)

#test.listExchanges()

#test.as_dict()
    

#test

#test.listExchanges()

#[exc.input for exc in agb.findActivity('syngas updated', 
#                 db_name = 'Foreground DB').exchanges()] #listExchanges()

In [6]:
process_tree(model, foreground_only=True)

In [5]:
agb.list_databases()

In [162]:
agb.list_parameters()

In [7]:
list_processes(model, foreground_only=True)

In [8]:
parameters = {
    'year': 2020,
    'model': 'remind',
    'pathway': 'SSP2_Base',
    #'energy_consumption_electrofuel': 10e6,
    #'elec_solar_share': 0.0
}

res = agb.compute_impacts(
    model,
    methods,
    return_params=False,
    #axis='phase',
    **parameters,
)
res

In [10]:
parameters = {
    'year': [year for year in range(2020, 2050)],
    'model': 'remind',
    'pathway': 'SSP2_Base'
    'energy_consumption_electrofuel': 10e6,
    'elec_solar_share': 0.0
}

res = agb.compute_impacts(
    model,
    methods,
    return_params=False,
    **parameters,
)
res

In [24]:
import time
start_time = time.time()

lambdas = agb.lca._preMultiLCAAlgebric(model, methods)

print("--- %s seconds ---" % (time.time() - start_time))

In [122]:
import time
start_time = time.time()

parameters = {
    'year': [year for year in range(2020, 2050)],
    'model': 'remind',
    'pathway': 'SSP2_Base'
}

dfs = []
param = {
        'year': 2020,
        'model': 'remind',
        'pathway': 'SSP2_Base'
    }

for i, year in enumerate(parameters):
    param['year'] = year
    res = agb.lca._postMultiLCAAlgebric(methods, lambdas, **parameters)
    dfs.append(res)
    #res = _postMultiLCAAlgebric(methods, lambdas, **parameters)

print("--- %s seconds ---" % (time.time() - start_time))

In [139]:
def compute_impacts_from_lambdas(
    lambdas,
    methods,
    axis='phase',
    **params: Dict[str, agb.SingleOrMultipleFloat],
):
    """
    Modified version of compute_impacts from lca_algebraic.
    More like a wrapper of _postLCAAlgebraic, to avoid calling _preLCAAlgebraic which is unecessarily time consuming when lambdas have already been calculated and doesn't have to be updated.
    """
    dfs = dict()

    # Gather all param values (even default and computed)
    params_all = dict()

    # Check no params are passed for FixedParams
    for key in params:
        if key in agb.params._fixed_params():
            warn("Param '%s' is marked as FIXED, but passed in parameters : ignored" % key)

    #lambdas = _preMultiLCAAlgebric(model, methods, alpha=alpha, axis=axis)

    df = agb.lca._postMultiLCAAlgebric(methods, lambdas, **params)

    model_name = agb.base_utils._actName(model)
    while model_name in dfs:
        model_name += "'"

    # param with several values
    list_params = {k: vals for k, vals in params.items() if isinstance(vals, list)}

    # Shapes the output / index according to the axis or multi param entry
    if axis:
        df[axis] = lambdas[0].axis_keys
        df = df.set_index(axis)
        df.index.set_names([axis])

        # Filter out line with zero output
        df = df.loc[
            df.apply(
                lambda row: not (row.name is None and row.values[0] == 0.0),
                axis=1,
            )
        ]

        # Rename "None" to others
        df = df.rename(index={None: "_other_"})

        # Sort index
        df.sort_index(inplace=True)

        # Add "total" line
        df.loc["*sum*"] = df.sum(numeric_only=True)

    elif len(list_params) > 0:
        for k, vals in list_params.items():
            df[k] = vals
        df = df.set_index(list(list_params.keys()))

    else:
        # Single output ? => give the single row the name of the model activity
        df = df.rename(index={0: model_name})

    dfs[model_name] = df

    if len(dfs) == 1:
        df = list(dfs.values())[0]
    else:
        # Concat several dataframes for several models
        df = pd.concat(list(dfs.values()))

    return df

In [158]:
start_time = time.time()
parameters = {
    'year': [year for year in range(2020, 2050)],
    'model': 'remind',
    'pathway': 'SSP2_Base',
    'energy_consumption_electrofuel': 10e12,
    'energy_consumption_kerosene': 0.0,
    'energy_consumption_biofuel': 0.0,
    'elec_solar_share': [1.0 for year in range(2020, 2050)]
}
res = compute_impacts_from_lambdas(lambdas, methods, axis=None, **parameters)
print("--- %s seconds ---" % (time.time() - start_time))

In [159]:
res

In [161]:
res['climate change - global warming potential (GWP100)[kg CO2-Eq]'].plot()

In [209]:
parameters = {
    'year': [year for year in range(2020, 2051)],
    'share_efuel': [i/2051 for i in range(2020, 2051)],
    'model': 'remind',
    'pathway': 'SSP2_Base'
}

# Create an empty DataFrame to store the results for each impact method and each year
multi_df_lca = pd.DataFrame()

for i, year in enumerate(parameters['year']):
    parameters_tmp = parameters.copy()
    for key, val in parameters_tmp.items():
        if isinstance(val, list):
            parameters_tmp[key] = val[i]

    res = agb.compute_impacts(
        model,
        methods,
        axis='phase',
        **parameters_tmp,
    )
    
    # Iterate over each method
    df_year = pd.DataFrame()  # DataFrame for the results of each impact method for the current year
    for method in res.columns:
        # Extract the data for the current method
        data = res[method]
        # Create a DataFrame with MultiIndex consisting of method and year
        df_year_method = pd.DataFrame(data.values, columns=[year], index=pd.MultiIndex.from_product([[method], data.index], names=['method', 'phase']))
        # Concatenate the new DataFrame with the existing DataFrame
        df_year = pd.concat([df_year, df_year_method], axis=0)
        
    multi_df_lca = pd.concat([multi_df_lca, df_year], axis=1)

multi_df_lca

In [221]:
test = get_parameter('aircraft_lifetime')
test.is_aeromaps_output = True
test.

In [222]:
test = list()
test += ['a', 'b']
test

In [196]:
# Assuming df is your DataFrame
series_list = []

# Iterate over each row of the DataFrame
for index, row in multi_df_lca.iterrows():
    # Convert the row to a Pandas Series
    series = row.squeeze()
    # Set the name of the series as the tuple index
    series.name = index
    # Append the series to the list
    series_list.append(series)

# Now series_list contains all the Pandas Series corresponding to each row of the MultiIndex DataFrame
#series_list

In [212]:
multi_df_lca.columns

In [215]:
test = pd.DataFrame(columns=list(range(2020, 2051)))
test.columns

In [9]:
agb.findActivity('kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation',  db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050', loc='World').listExchanges()

In [10]:
def sum_amounts(act, exchange_name):
    exchs = act.getExchange(exchange_name, single=False)
    total = sum(exch['amount'] for exch in exchs)
    return total

act = agb.findActivity('kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation',  db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050', loc='World')#.listExchanges()
exchange_name = 'market group for electricity*'
sum_amounts(act, exchange_name)